## Using Variational Estimates to Initialize the NUTS-HMC Sampler

In this example we show how to use the parameter estimates return by Stan's variational inference algorithm
as the initial parameter values for Stan's NUTS-HMC sampler, using a the [earnings-logearn_height model](https://github.com/stan-dev/posteriordb/blob/master/posterior_database/models/stan/logearn_height.stan) and data from the [posteriordb package](https://github.com/stan-dev/posteriordb).

The experiments reported in the paper [Pathfinder: Parallel quasi-Newton variational inference](https://arxiv.org/abs/2108.03782) by Zhang et al. show that mean-field ADVI provides a better estimate of the posterior, as measured by the 1-Wasserstein distance to the reference posterior, than 75 iterations of the warmup Phase I algorithm used by the NUTS-HMC sampler, furthermore, ADVI is more computationally efficient, requiring fewer evaluations of the log density and gradient functions.  Therefore, using the estimates from ADVI to initialize the parameter values for the NUTS-HMC sampler will allow the sampler to do a better job of adapting the stepsize and metric during warmup, resulting in better performance and estimation.


### Model and data

For conveince, we have copied the posteriordb model and data to this directory, in files [logearn_height.stan](logearn_height.stan) and [earnings.json](earnings.json).

In [ ]:
import os
from cmdstanpy import CmdStanModel
    
stan_file = 'logearn_height.stan'
data_file = 'earnings.json'

# instantiate, compile bernoulli model
model = CmdStanModel(stan_file=stan_file)

### Run Stan's variational inference algorithm, obtain fitted estimates

The `CmdStanModel` method `variational` runs CmdStan's ADVI algorithm.
Conditioning the model on the data results in a posterior geometry which is difficult to navigate.  Because the ADVI algorithm may fail to converge, we run it with argument `require_converged` set to `False`.

In [ ]:
vb_fit = model.variational(data=data_file, require_converged=False, seed=123)

The ADVI algorithm provides estimates of all model parameters as well as the step size scaling factor `eta`.

The `variational` method returns a `CmdStanVB` object, with methods `eta` and `stan_variables`, which
return the step size scaling factor and estimates of all model parameters as a Python dictionary respectively.

In [ ]:
print(vb_fit.eta, vb_fit.stan_variables())

Posteriordb provides reference posteriors for all models.  For the logearn_height model, conditioned on the dataset `earnings.json`, the posterior variables are:

- beta[1]:  5.782
- beta[2]:  0.059
- sigma: 0.894

By default, the sampler algorithm randomly initializes all model parameters in the range uniform[-2, 2].  The ADVI estimates will provide a better starting point, especially w/r/t to parameter `beta[1]`, than the defaults.
In addition, we can use the step size scaling factor to scale the initial step size, which allows us to skip the first phase of warmup (default 75 iterations).

In [ ]:
vb_vars = vb_fit.stan_variables()
vb_stepsize = 1.0 / vb_fit.eta
mcmc_vb_inits_fit = model.sample(
    data=data_file, inits=vb_vars, step_size=vb_stepsize,
    adapt_init_phase=0, seed=123
)

In [ ]:
mcmc_vb_inits_fit.summary()

The sampler results match the reference posterior, (taking into account MCSE).

- beta[1]:  5.782
- beta[2]:  0.059
- sigma: 0.894

To see how this is useful, we run the sampler with default initializations, step size, and warmup, (same random seed).

In [ ]:
mcmc_random_inits_fit = model.sample(data=data_file, seed=123)

In [ ]:
mcmc_random_inits_fit.summary()

On my machine, using the variational estimates to skip warmup phase I shows improved N_Eff/s (number of effective sampler per second) values for all parameters.

This is a simple model run on a small dataset.  For complex models where the initial parameter values are far from the default initializations, this procedure may allow for faster and better adaptation during warmup.